## Creating a TF record 
This record is a data format needed by the tnesor flow YOLOV3 API

## Imports & Basic Cifig

In [1]:
#%tensorflow_version 2.x


In [2]:
#import tensorflow
#print(tensorflow.__version__)

In [3]:
#import keras
#print(keras.__version__)

In [4]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import sys
import numpy as np
import seaborn as sns
import statistics as stats
sns.set(color_codes=True)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
#Last import allows multiple outputs from one cell
import warnings
# Initialize the random number generator
import random
random.seed(0)

#Additional libs
import matplotlib.patches as patches


In [5]:
import os
import tensorflow 
from keras.models import Model
#from tensorflow.keras.optimizers import Adam, SGD
from keras.applications.vgg16 import VGG16, preprocess_input
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Dropout, Flatten
from pathlib import Path
import numpy as np
import matplotlib.image as img

#import tensorflow_addons as tfa

In [6]:
# Useful Configuration/Setting

# suppress display of warnings
#warnings.filterwarnings('ignore')

# display all dataframe columns
pd.options.display.max_columns = None

# to set the limit to 3 decimals
pd.options.display.float_format = '{:.7f}'.format

# display all dataframe rows
pd.options.display.max_rows = None

#Setting to shows all entries in array displayed
np.set_printoptions(threshold=sys.maxsize)

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Create TF Record Format

In [8]:
train_data_dir = Path('/content/drive/MyDrive/Capstone/Car Images/Car Images/Train Images')
#test_data_dir = Path('/content/drive/MyDrive/Capstone/Car Images/Car Images/Test Images')

In [9]:
import pandas as pd

#Lets read the Train Annotations

#Lets give our own column names as excel sheet does not have proper column names
columnNameList=['file_name','cor1','cor2','cor3','cor4','Image_Class']
trainAnno= pd.read_csv('/content/drive/MyDrive/Capstone/Annotations/Train Annotations.csv',names=columnNameList,header=0)
trainAnno.head(5)

,file_name,cor1,cor2,cor3,cor4,Image_Class
0,00001.jpg,39,116,569,375,14
1,00002.jpg,36,116,868,587,3
2,00003.jpg,85,109,601,381,91
3,00004.jpg,621,393,1484,1096,134
4,00005.jpg,14,36,133,99,106


In [10]:
files = train_data_dir.rglob('*.jpg')
input=pd.DataFrame(files,columns =['a'])
input['a']=input['a'].astype(str)
df_train=input['a'].str.split('/',expand=True)
train=input.join(df_train)[['a',8]]
train.columns=('file_path','label')

In [11]:
train.head()
len(train)

,file_path,label
0,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012
1,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012
2,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012
3,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012
4,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012


8144

In [12]:
train.head()

,file_path,label
0,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012
1,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012
2,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012
3,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012
4,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012


In [13]:
train['file_name']=train['file_path'].apply(lambda getFileName:getFileName.split('/')[9])

In [14]:
train.head()

,file_path,label,file_name
0,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012,00482.jpg
1,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012,05872.jpg
2,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012,06576.jpg
3,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012,00719.jpg
4,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012,04655.jpg


In [15]:
trainAnno.head()

,file_name,cor1,cor2,cor3,cor4,Image_Class
0,00001.jpg,39,116,569,375,14
1,00002.jpg,36,116,868,587,3
2,00003.jpg,85,109,601,381,91
3,00004.jpg,621,393,1484,1096,134
4,00005.jpg,14,36,133,99,106


In [16]:
# Lets add coordinate information to the above table
trainDF=pd.merge(train,trainAnno,on='file_name',how='inner')

In [17]:
trainDF.head(5)

,file_path,label,file_name,cor1,cor2,cor3,cor4,Image_Class
0,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012,00482.jpg,80,91,589,429,184
1,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012,05872.jpg,39,35,591,413,184
2,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012,06576.jpg,64,347,629,577,184
3,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012,00719.jpg,118,93,380,317,184
4,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012,04655.jpg,9,11,824,489,184


In [18]:
# We need to now convert this to tf record specific format

# Lets take a copy first
trainDFTFRecord=trainDF.copy()

In [19]:
# Lets create a dict mapping of text labels :Class
# We will not use the existing "Image_Class" but assign new ones

In [20]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(trainDFTFRecord['label'])

#le.classes_

trainDFTFRecord['Imageclass']=le.transform(trainDFTFRecord['label'])
#trainDFTFRecord.sample(10)

LabelEncoder()

In [21]:
# We need to add 1 to all labels because OD API expects index to start from 1 and not 0
trainDFTFRecord['Imageclass']=trainDFTFRecord['Imageclass']+1

In [22]:
trainDFTFRecord.drop(labels=['Image_Class'], axis=1,inplace=True)

In [23]:
trainDFTFRecord.sample(5)

,file_path,label,file_name,cor1,cor2,cor3,cor4,Imageclass
4688,/content/drive/MyDrive/Capstone/Car Images/Car...,Lamborghini Aventador Coupe 2012,07978.jpg,68,238,872,525,150
4383,/content/drive/MyDrive/Capstone/Car Images/Car...,Daewoo Nubira Wagon 2002,06762.jpg,23,24,237,177,82
1990,/content/drive/MyDrive/Capstone/Car Images/Car...,Ford Mustang Convertible 2007,07655.jpg,16,18,249,159,116
1313,/content/drive/MyDrive/Capstone/Car Images/Car...,Chevrolet Traverse SUV 2012,04137.jpg,199,482,3385,2465,75
6647,/content/drive/MyDrive/Capstone/Car Images/Car...,Ford Expedition EL SUV 2009,06419.jpg,26,16,244,164,108


In [24]:
#We need a dictionary as an arg to create_tf_record
mapping = dict(zip(le.classes_, range(0,len(le.classes_))))

In [25]:
#mapping

In [26]:
# Renaming accorinf to tf_record
trainDFTFRecord.rename(columns = {'file_name':'ImageName','label':'ImageClassName','cor1':'x0','cor2':'y0','cor3':'x1','cor4':'y1'}, inplace = True)

In [27]:
trainDFTFRecord.sample(5)

,file_path,ImageClassName,ImageName,x0,y0,x1,y1,Imageclass
1189,/content/drive/MyDrive/Capstone/Car Images/Car...,GMC Savana Van 2012,05199.jpg,224,49,621,346,120
7052,/content/drive/MyDrive/Capstone/Car Images/Car...,Dodge Caliber Wagon 2007,07737.jpg,17,43,290,201,83
498,/content/drive/MyDrive/Capstone/Car Images/Car...,Mercedes-Benz SL-Class Coupe 2009,04675.jpg,35,47,464,228,165
6370,/content/drive/MyDrive/Capstone/Car Images/Car...,Ford Focus Sedan 2007,05970.jpg,51,164,602,359,113
2193,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki Kizashi Sedan 2012,07400.jpg,130,70,1169,806,182


In [28]:
from sklearn.model_selection import train_test_split
train_df,valid_df= train_test_split(trainDFTFRecord, test_size=0.20, random_state=0, stratify=train['label'])

In [29]:
#train_df,valid_df= train_test_split(valid_df, test_size=0.20, random_state=1, stratify=valid_df['label'])

In [30]:
train_df.shape
valid_df.shape

(6515, 8)

(1629, 8)

In [52]:
train_df.head(5)
valid_df.head(5)

,file_path,ImageClassName,ImageName,x0,y0,x1,y1,Imageclass
7925,/content/drive/MyDrive/Capstone/Car Images/Car...,BMW M6 Convertible 2010,03091.jpg,35,34,419,297,34
6702,/content/drive/MyDrive/Capstone/Car Images/Car...,Honda Accord Coupe 2012,04995.jpg,26,84,401,284,126
2089,/content/drive/MyDrive/Capstone/Car Images/Car...,Dodge Charger Sedan 2012,07012.jpg,36,52,553,320,88
7543,/content/drive/MyDrive/Capstone/Car Images/Car...,Ford Fiesta Sedan 2012,07955.jpg,21,56,623,376,112
3998,/content/drive/MyDrive/Capstone/Car Images/Car...,BMW X6 SUV 2012,00930.jpg,31,89,296,187,37


,file_path,ImageClassName,ImageName,x0,y0,x1,y1,Imageclass
2733,/content/drive/MyDrive/Capstone/Car Images/Car...,Chrysler Sebring Convertible 2010,02306.jpg,29,95,557,375,80
4421,/content/drive/MyDrive/Capstone/Car Images/Car...,Daewoo Nubira Wagon 2002,07283.jpg,108,95,621,392,82
6320,/content/drive/MyDrive/Capstone/Car Images/Car...,Buick Verano Sedan 2012,02450.jpg,51,99,595,382,50
371,/content/drive/MyDrive/Capstone/Car Images/Car...,Volvo XC90 SUV 2007,02332.jpg,65,92,606,420,195
3937,/content/drive/MyDrive/Capstone/Car Images/Car...,Chrysler 300 SRT-8 2010,03784.jpg,84,150,1029,616,76


In [32]:
% pwd

'/content'

In [33]:
% ls

drive/  sample_data/


In [34]:
print(mapping)

{'AM General Hummer SUV 2000': 0, 'Acura Integra Type R 2001': 1, 'Acura RL Sedan 2012': 2, 'Acura TL Sedan 2012': 3, 'Acura TL Type-S 2008': 4, 'Acura TSX Sedan 2012': 5, 'Acura ZDX Hatchback 2012': 6, 'Aston Martin V8 Vantage Convertible 2012': 7, 'Aston Martin V8 Vantage Coupe 2012': 8, 'Aston Martin Virage Convertible 2012': 9, 'Aston Martin Virage Coupe 2012': 10, 'Audi 100 Sedan 1994': 11, 'Audi 100 Wagon 1994': 12, 'Audi A5 Coupe 2012': 13, 'Audi R8 Coupe 2012': 14, 'Audi RS 4 Convertible 2008': 15, 'Audi S4 Sedan 2007': 16, 'Audi S4 Sedan 2012': 17, 'Audi S5 Convertible 2012': 18, 'Audi S5 Coupe 2012': 19, 'Audi S6 Sedan 2011': 20, 'Audi TT Hatchback 2011': 21, 'Audi TT RS Coupe 2012': 22, 'Audi TTS Coupe 2012': 23, 'Audi V8 Sedan 1994': 24, 'BMW 1 Series Convertible 2012': 25, 'BMW 1 Series Coupe 2012': 26, 'BMW 3 Series Sedan 2012': 27, 'BMW 3 Series Wagon 2012': 28, 'BMW 6 Series Convertible 2007': 29, 'BMW ActiveHybrid 5 Sedan 2012': 30, 'BMW M3 Coupe 2012': 31, 'BMW M5 Sed

In [35]:
#Build a pbtxt label file using label and class name
#This is required by Object detection API
#You can prepare it manually as well.
key_list = list(mapping.keys())
val_list = list(mapping.values())

pbtxt_file_txt = ''
for numlabel in val_list:
  # We need to add 1 to each label because 0 is reserved
  pbtxt_file_txt += "item {\n  id: " + str(numlabel +1) + "\n  name: '" + key_list[val_list.index(numlabel)]  + "'\n}\n\n"

with open('/content/drive/MyDrive/Capstone_Rework/YOLOV3/TF_Files/label_map.txt','w') as pbfile:
    pbfile.write(pbtxt_file_txt)

11226

In [36]:
train_df.to_csv('/content/drive/MyDrive/Capstone_Rework/YOLOV3/TF_Files/train_df.csv', index=False)
valid_df.to_csv('/content/drive/MyDrive/Capstone_Rework/YOLOV3/TF_Files/valid_df.csv', index=False)

## Creating the tf_record

In [37]:
! git clone https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 71201, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 71201 (delta 23), reused 29 (delta 18), pack-reused 71165
Receiving objects: 100% (71201/71201), 578.63 MiB | 48.07 MiB/s, done.
Resolving deltas: 100% (50324/50324), done.


In [38]:
%%bash
cd models/research
# Compile protos.
protoc object_detection/protos/*.proto --python_out=.

# Install TensorFlow Object Detection API.
cp object_detection/packages/tf2/setup.py .
python -m pip install --use-feature=2020-resolver .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1686356 sha256=a5497f6d40d63f8b1a1a2b89d04134a15a1ae12bb15bcfa94201d6d6cdda87f5
  Stored in directory: /tmp/pip-ephem-wheel-cache-yppd0rvs/wheels/fa/a4/d2/e9a5057e414fd46c8e543d2706cd836d64e1fcd9eccceb2329
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-py3-none-any.whl size=22257 sha256=3a511c519d4110ef92274386579abfd988dde7afb6ad0a60878bc36fe79ba1e3
  Stored in directory: /root/.cache/pip/wheels/d2/f1/1f/041add21dc9c4220157f1bd2bd6afe1f1a49524c3396b94401
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78544 sha256=2ab5550c8ca47f548d65d2b74fb437e65bbc3b743dc231282c6dec22cf5c3b16
  Stored in directory: /root/.cache/pip/wheels/a4/61/fd/c57e374e580aa78a45ed78d5859b3a44436af17e22ca53284f
  Created wheel for avro-python3: filename=avro_python3-1.10.2-py3-none-any.whl size=44010 sha256=521656527fdaeb4395becd1d853a4697505805deb10b0fb028

  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
multiprocess 0.70.12.2 requires dill>=0.3.4, but you have dill 0.3.1.1 which is incompatible.
gym 0.17.3 requires cloudpickle<1.7.0,>=1.2.0, but you have cloudpickle 2.0.0 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [39]:
#! cd /content/drive/MyDrive/Capstone_Rework/YOLOV3/TF_Files/

In [40]:
#! python '/content/models/research/object_detection/dataset_tools/create_pascal_tf_record.py' --year=merged --annotations_dir=train_df.csv --output_path=train.tfrecord --label_map_path='/content/drive/MyDrive/Capstone_Rework/YOLOV3/TF_Files/label_map.txt'

In [49]:
# Re-declaring variables for clarity:

train_data_dir = '/content/drive/MyDrive/Capstone/Car Images/Car Images/Train Images'
TF_Train_Op_Dir='/content/drive/MyDrive/Capstone_Rework/YOLOV3/TF_Files/trainDF.tfrecord'

In [50]:
train_data_dir

'/content/drive/MyDrive/Capstone/Car Images/Car Images/Train Images'

In [42]:
% pwd

'/content'

In [43]:
% cd /content/drive/MyDrive/Capstone_Rework/YOLOV3/TF_Files/

/content/drive/MyDrive/Capstone_Rework/YOLOV3/TF_Files


In [54]:
#generate tfrecord for training data
!python generate_tfrecord.py  --csv_input=train_df.csv  --img_path={train_data_dir} --output_path={TF_Train_Op_Dir}

2022-03-25 07:57:11.101906: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Successfully created the TFRecords: /content/drive/MyDrive/Capstone_Rework/YOLOV3/TF_Files/trainDF.tfrecord


In [ ]:
TF_Train_Op_Val_Dir='/content/drive/MyDrive/Capstone_Rework/YOLOV3/TF_Files/valDF.tfrecord'

In [55]:
#generate tfrecord for training data
!python generate_tfrecord.py  --csv_input=valid_df.csv  --img_path={train_data_dir} --output_path={TF_Train_Op_Val_Dir}

2022-03-25 08:27:19.728593: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Successfully created the TFRecords: /content/drive/MyDrive/Capstone_Rework/YOLOV3/TF_Files/{TF_Train_Op_Val_Dir}
